## 1. Librerias necesarias

In [6]:
import findspark 
findspark.init('/opt/spark')

import pyspark 
sc = pyspark.SparkContext(appName = "myAppName")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=myAppName, master=local[*]) created by __init__ at <ipython-input-1-e7a57a5858a2>:5 

In [8]:
from pyspark import SparkContext,HiveContext
from pyspark.sql import SQLContext,SparkSession
from pyspark.sql import functions as F
import os
import subprocess
import pandas as pd
from datetime import date

## 2. Listado de archivos desde hdfs

In [14]:
ruta = 'hdfs://localhost:9000/user/hive/warehouse/nyc_taxi_data_2014.csv'

## 3. Lectura de archivos desde HDFS a Spark

In [9]:
sparkSession = SparkSession.builder.appName("myAppName").getOrCreate()

In [15]:
df = sparkSession.read.csv(ruta,inferSchema=False,header=True)

In [16]:
type(df)

pyspark.sql.dataframe.DataFrame

In [17]:
df.printSchema()

root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- total_amount: string (nullable = true)



In [18]:
%%time
df.count()

CPU times: user 25.2 ms, sys: 9.81 ms, total: 35 ms
Wall time: 2min 11s


14999999

In [19]:
%%time
df = df.withColumn('trip_start_timestamp',F.to_timestamp(F.col('trip_start_timestamp')))
df = df.withColumn('trip_end_timestamp',F.to_timestamp(F.col('trip_end_timestamp')))

CPU times: user 3.87 ms, sys: 271 µs, total: 4.14 ms
Wall time: 39.2 ms


In [23]:
var = ['taxi_id',
       'trip_start_timestamp',
       'trip_seconds',
       'trip_miles',
       'trip_total',
      'payment_type']

In [25]:
df = df.select(*var)

In [26]:
df.printSchema()

root
 |-- taxi_id: string (nullable = true)
 |-- trip_start_timestamp: timestamp (nullable = true)
 |-- trip_seconds: string (nullable = true)
 |-- trip_miles: string (nullable = true)
 |-- trip_total: string (nullable = true)
 |-- payment_type: string (nullable = true)



In [27]:
for v in df.columns:
    if v in ['trip_seconds','trip_miles','trip_total']:
        df = df.withColumn(v,F.col(v).cast('double'))

In [28]:
df.printSchema()

root
 |-- taxi_id: string (nullable = true)
 |-- trip_start_timestamp: timestamp (nullable = true)
 |-- trip_seconds: double (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_total: double (nullable = true)
 |-- payment_type: string (nullable = true)



## 4. Operaciones básicas

In [29]:
%%time
df.count()

CPU times: user 4.42 ms, sys: 315 µs, total: 4.74 ms
Wall time: 15.7 s


19866157

In [30]:
df.select('taxi_id','trip_total').show(6)

+-------+----------+
|taxi_id|trip_total|
+-------+----------+
|     85|       4.5|
|   2776|       8.9|
|   3168|     47.75|
|   4237|       7.0|
|   5710|     10.25|
|   1987|     17.75|
+-------+----------+
only showing top 6 rows



In [32]:
%%time
df.describe('trip_total').show()

+-------+------------------+
|summary|        trip_total|
+-------+------------------+
|  count|          19865857|
|   mean| 16.62857499528094|
| stddev|34.139991307069685|
|    min|               0.0|
|    max|            9999.0|
+-------+------------------+

CPU times: user 4.93 ms, sys: 61 µs, total: 5 ms
Wall time: 7.61 s


In [33]:
%%time
df2=df.filter((df.trip_total>100)&(df['payment_type']=='Credit Card'))

CPU times: user 4.33 ms, sys: 0 ns, total: 4.33 ms
Wall time: 31.3 ms


In [34]:
%%time
df2.count()

CPU times: user 4.32 ms, sys: 317 µs, total: 4.63 ms
Wall time: 17.2 s


36045

In [35]:
%%time
df2.select('taxi_id','trip_total','payment_type').show(5)

+-------+----------+------------+
|taxi_id|trip_total|payment_type|
+-------+----------+------------+
|   6172|    102.75| Credit Card|
|   7555|    103.75| Credit Card|
|   3788|     114.0| Credit Card|
|   5254|    190.45| Credit Card|
|    392|     159.0| Credit Card|
+-------+----------+------------+
only showing top 5 rows

CPU times: user 0 ns, sys: 5.69 ms, total: 5.69 ms
Wall time: 3.07 s


## 5. Funciones definidas por el usuario

In [36]:
func_mes = F.udf(lambda x:int(x.strftime('%Y%m')))

In [37]:
%%time
df.withColumn('mes',func_mes(F.col('trip_start_timestamp'))).select('trip_start_timestamp','mes').show(5)

+--------------------+------+
|trip_start_timestamp|   mes|
+--------------------+------+
| 2016-01-13 06:15:00|201601|
| 2016-01-22 09:30:00|201601|
| 2016-01-31 21:30:00|201601|
| 2016-01-23 17:30:00|201601|
| 2016-01-14 05:45:00|201601|
+--------------------+------+
only showing top 5 rows

CPU times: user 8.8 ms, sys: 4.45 ms, total: 13.3 ms
Wall time: 2.24 s


In [38]:
%%time
df = df.withColumn('mes',func_mes(F.col('trip_start_timestamp')).cast('integer'))

CPU times: user 3.63 ms, sys: 0 ns, total: 3.63 ms
Wall time: 12.1 ms


In [39]:
df.printSchema()

root
 |-- taxi_id: string (nullable = true)
 |-- trip_start_timestamp: timestamp (nullable = true)
 |-- trip_seconds: double (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_total: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- mes: integer (nullable = true)



In [41]:
%%time

expr = [y(x).alias("v_%s_%s"%(x,z)) for x in ['trip_total','trip_miles','trip_seconds'] for y,z in zip([F.min,F.mean,F.max],
                                                               ['minimo','media','maximo'])]
df.groupby('mes').agg(*expr).show()

+------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+---------------------+--------------------+---------------------+
|   mes|v_trip_total_minimo|v_trip_total_media|v_trip_total_maximo|v_trip_miles_minimo|v_trip_miles_media|v_trip_miles_maximo|v_trip_seconds_minimo|v_trip_seconds_media|v_trip_seconds_maximo|
+------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+---------------------+--------------------+---------------------+
|201609|                0.0|17.133133092684762|             9550.5|                0.0|3.1642990042489623|              932.4|                  0.0|   847.7482997666043|              86399.0|
|201604|                0.0|16.760364944398116|            9901.05|                0.0| 4.035661179188304|             3300.0|                  0.0|   732.3746028426701|              86388.0|
|201611|                0.0| 16.96932620

## 6. Uso de SQL 

In [42]:
sqlContext = SQLContext(sc)

In [43]:
df.registerTempTable("chicago")

In [46]:
query = """
SELECT year(trip_start_timestamp)*100+month(trip_start_timestamp) AS FH, 
COUNT(*) AS CASOS FROM chicago GROUP BY 
year(trip_start_timestamp)*100+month(trip_start_timestamp)  ORDER BY FH
"""

In [47]:
%%time
sqlContext.sql(query).show()

+------+-------+
|    FH|  CASOS|
+------+-------+
|201601|1705805|
|201602|1751192|
|201603|1975108|
|201604|1952152|
|201605|1943584|
|201606|1934479|
|201607|1745387|
|201608|1523947|
|201609|1294020|
|201610|1499771|
|201611|1295000|
|201612|1245712|
+------+-------+

CPU times: user 4.84 ms, sys: 371 µs, total: 5.21 ms
Wall time: 23 s


In [48]:
tabla = sqlContext.sql(query)
type(tabla)

pyspark.sql.dataframe.DataFrame

## 7. Leer tabla desde Hive

In [49]:
%%time
hive_context = HiveContext(sc)
tabla = hive_context.table("taxis.chicago")

CPU times: user 296 µs, sys: 4.17 ms, total: 4.46 ms
Wall time: 146 ms


In [50]:
type(tabla)

pyspark.sql.dataframe.DataFrame

In [51]:
%%time
tabla.count()

CPU times: user 496 µs, sys: 3.82 ms, total: 4.31 ms
Wall time: 18.3 s


19866169

In [52]:
%%time
tabla = tabla.groupby('payment_type').agg({'trip_total':'mean'})

CPU times: user 4.22 ms, sys: 6 µs, total: 4.22 ms
Wall time: 18.2 ms


In [53]:
tabla.show(4)

+------------+------------------+
|payment_type|   avg(trip_total)|
+------------+------------------+
| Credit Card| 21.34001686077414|
|   No Charge|18.013121714062898|
|     Unknown|11.577861055469077|
|      Prcard|11.211451040555353|
+------------+------------------+
only showing top 4 rows



## 8. Escribir en Hive y HDFS

In [56]:
tabla.createOrReplaceTempView('grupo_taxis')

In [57]:
%%time
hive_context.sql('create table taxis.grupo as select * from grupo_taxis')

CPU times: user 4.95 ms, sys: 386 µs, total: 5.33 ms
Wall time: 23.5 s


DataFrame[]

In [60]:
#hive_context.sql('create table taxis.xxx as select taxi_id from chicago')

In [64]:
%%time
'hdfs://%s'%ruta
tabla.write.format('csv').save('hdfs://%s/agrupado.csv'%ruta)
#tabla.write.format('parquet').save('hdfs://%s/agrupado.parquet'%ruta)

CPU times: user 4.63 ms, sys: 35 µs, total: 4.66 ms
Wall time: 21.5 s


In [66]:
df3 = sparkSession.read.csv('hdfs://%s/agrupado.csv'%ruta,header=True,inferSchema=False)
df3.count()

0

In [68]:
df3.show()

+-----------+-----------------+
|Credit Card|21.34001686077414|
+-----------+-----------------+
+-----------+-----------------+



## 9. Cruces

In [71]:
%%time
df =  hive_context.table("taxis.grupo")
df2 = df.filter(df['payment_type']=='Credit Card')

CPU times: user 0 ns, sys: 2.85 ms, total: 2.85 ms
Wall time: 48.7 ms


In [72]:
df.show()

+------------+------------------+
|payment_type|   avg(trip_total)|
+------------+------------------+
| Credit Card| 21.34001686077414|
|   No Charge|18.013121714062898|
|     Unknown|11.577861055469077|
|      Prcard|11.211451040555353|
|        Cash|12.465961928839715|
|     Dispute|15.169767331415997|
|    Way2ride| 5.160909078337929|
|       Pcard|10.120076191856748|
|payment_type|              null|
+------------+------------------+



In [73]:
df2.show()

+------------+-----------------+
|payment_type|  avg(trip_total)|
+------------+-----------------+
| Credit Card|21.34001686077414|
+------------+-----------------+



In [75]:
%%time
df = df.join(df2, ['payment_type'],'inner')

CPU times: user 3.08 ms, sys: 0 ns, total: 3.08 ms
Wall time: 14.7 ms


## 10. Convertir a pandas

In [76]:
%%time
df = df.toPandas()

CPU times: user 13.1 ms, sys: 3.75 ms, total: 16.8 ms
Wall time: 1 s


In [78]:
df

,payment_type,avg(trip_total),avg(trip_total)
0,Credit Card,21.340017,21.340017


In [77]:
df.describe()

,avg(trip_total),avg(trip_total)
count,1.000000,1.000000
mean,21.340017,21.340017
std,NaN,NaN
min,21.340017,21.340017
25%,21.340017,21.340017
50%,21.340017,21.340017
75%,21.340017,21.340017
max,21.340017,21.340017
